# Data Exploration and Analysis

## 1. Reading the data

If the data is in the zip file format then use the below code, else skip this section

In [2]:
# Importing libraries
import os
import zipfile
import requests

# Defining the function to dowload the data
def download_data(url, name, path='data'):
    
    if not os.path.exists(path):
        os.mkdir(path)
    response = requests.get(url)
    
    with open(os.path.join(path, name), 'wb') as f:
        f.write(response.content)
    z = zipfile.ZipFile(os.path.join(path, name))
    z.extractall(path)

In [3]:
# Calling the above function as an example
VEHICLES = 'http://bit.ly/ddl-cars'
download_data(VEHICLES, 'vehicles.zip')

If you already have extracted files in csv, excel, json or any other format then you can directly load the data in pandas dataframe by skipping the above step

In [4]:
# loading the data in pandas dataframe
import pandas as pd

path = 'data'
vehicles = pd.read_csv(os.path.join(path, 'vehicles.csv'))

C:\Users\Akanksha_PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<b>Let's check how the data looks like and summarize it</b>

In [5]:
#print the data size in rows x columns
vehicles.shape

(37843, 83)

83 columns is a big number, i am sure not all columns will be of our use. So, later i am gonna reduce it. Number of rows are quite less in number so that's good for beginner's.<br/>
Let's check out what all columns do we have and which one we can get rid of.

In [8]:
vehicles.columns

Index(['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08',
       'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2',
       'co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U',
       'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'cylinders',
       'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08',
       'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U', 'highwayCD',
       'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4', 'make',
       'model', 'mpgData', 'phevBlended', 'pv2', 'pv4', 'range', 'rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'VClass', 'year', 'youSaveSpend', 'guzzler',
       'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'fuelType2', 'rangeA',
       'evMotor', 'mfrCode', 'c240Dscr', 'charge240b', 'c240bDscr',
       'createdOn', 'modifiedOn

In [10]:
vehicles.dtypes

barrels08          float64
barrelsA08         float64
charge120          float64
charge240          float64
city08               int64
city08U            float64
cityA08              int64
cityA08U           float64
cityCD             float64
cityE              float64
cityUF             float64
co2                  int64
co2A                 int64
co2TailpipeAGpm    float64
co2TailpipeGpm     float64
comb08               int64
comb08U            float64
combA08              int64
combA08U           float64
combE              float64
combinedCD         float64
combinedUF         float64
cylinders          float64
displ              float64
drive               object
engId                int64
eng_dscr            object
feScore              int64
fuelCost08           int64
fuelCostA08          int64
                    ...   
rangeCity          float64
rangeCityA         float64
rangeHwy           float64
rangeHwyA          float64
trany               object
UCity              float64
U

In [12]:
vehicles.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
barrels08,37843.0,17.532506,4.575950,0.06,14.33087,17.347895,20.600625,47.087143
barrelsA08,37843.0,0.216169,1.141527,0.00,0.00000,0.000000,0.000000,18.311667
charge120,37843.0,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
charge240,37843.0,0.023531,0.427647,0.00,0.00000,0.000000,0.000000,12.000000
city08,37843.0,17.941389,6.660360,6.00,15.00000,17.000000,20.000000,138.000000
city08U,37843.0,4.042737,9.645820,0.00,0.00000,0.000000,0.000000,138.304000
cityA08,37843.0,0.520149,3.837874,0.00,0.00000,0.000000,0.000000,127.000000
cityA08U,37843.0,0.327163,3.542596,0.00,0.00000,0.000000,0.000000,127.093000
cityCD,37843.0,0.000406,0.039918,0.00,0.00000,0.000000,0.000000,5.350000
cityE,37843.0,0.184790,2.904558,0.00,0.00000,0.000000,0.000000,122.000000


In [15]:
# Get number of missing values for columns
vehicles.isnull().sum()
# or use
#df.isnull().sum(axis = 0)

# If you want to get number of missing values for rows by any chance
#df.isnull().sum(axis = 1)

make              0
model             0
year              0
displ             0
cylinders         0
trany             0
drive             0
VClass            0
fuelType          0
barrels08         0
city08            0
highway08         0
comb08            0
co2TailpipeGpm    0
fuelCost08        0
dtype: int64

In [13]:
# Select required columns
select_columns = ['make', 'model', 'year', 'displ', 'cylinders', 'trany', 'drive', 'VClass','fuelType','barrels08', 'city08'
                  , 'highway08', 'comb08', 'co2TailpipeGpm', 'fuelCost08']
#vehicles = vehicles[select_columns][vehicles.year <= 2016].drop_duplicates().dropna()
vehicles = vehicles[select_columns].sort_values(['make', 'model', 'year']).drop_duplicates().dropna()
vehicles.head()

,make,model,year,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,comb08,co2TailpipeGpm,fuelCost08
19316,AM General,DJ Po Vehicle 2WD,1984,2.5,4.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,19.388824,18,17,17,522.764706,1950
19314,AM General,FJ8c Post Office,1984,4.2,6.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,25.354615,13,13,13,683.615385,2550
358,AM General,Post Office DJ5 2WD,1985,2.5,4.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,20.600625,16,17,16,555.437500,2100
369,AM General,Post Office DJ8 2WD,1985,4.2,6.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,25.354615,13,13,13,683.615385,2550
25797,ASC Incorporated,GNX,1987,3.8,6.0,Automatic 4-spd,Rear-Wheel Drive,Midsize Cars,Premium,20.600625,14,21,16,555.437500,2550


In [ ]:
# !!! DO NOT RUN THIS CELL UNTILL NEEDED !!!

# In case, if number of columns to drop is less in number then it is good idea to list down the columns to drop instead of 
# selecting columns

del_col_list = ['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08',
       'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2',
       'co2A']
vehicles_with_drop = vehicles.drop(del_col_list, axis=1)

In [ ]:
# !!! DO NOT RUN THIS CELL UNTILL NEEDED !!!

# Sometimes we have 0 instead of null or missing values so we need to first replace it with np.nan and then remove it from df

cols = ['co2TailpipeGpm', 'fuelCost08']
vehicles[cols] = vehicles[cols].replace(0, np.nan)
vehicles.dropna(subset=cols, inplace=True)